In [ ]:
!pip install langchain langchain-google-genai -U

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import time


# Configure API key
import os
from getpass import getpass

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google Gemini API key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)




ModuleNotFoundError: No module named 'langchain_google_genai'

In [ ]:

# 2A) Report prompt (markdown-ready)
report_prompt = PromptTemplate.from_template(
    """You are a public-health analyst.
Write a detailed, well-structured **markdown report** on the topic: "{topic}".

Requirements:
- Audience: intelligent non-experts (10th–12th grade clarity)
- Sections (use H2/H3): Introduction, Prevalence & Trends, Key Drivers, Health & Economic Impact, Policy & City-Level Interventions, Case Snapshots, Data Gaps, Conclusion
- Include India-urban context (income strata, food environment, sedentary work, women & children, metros vs tier-2)
- Be balanced, evidence-informed (no citations needed), ~800–1000 words
- End with a short “Key Terms” glossary

Return only markdown text."""
)

# 2B) Assemble the report chain:
# PromptTemplate -> ChatGoogleGenerativeAI -> StrOutputParser
report_chain = report_prompt | llm | StrOutputParser()

# response  = report_chain.invoke({"topic": "Obesity in Urban India"})
# print(response, "response")

In [ ]:
# Step 3: PromptTemplate + LCEL chain for the 5-point summary

summary_prompt = PromptTemplate.from_template(
    """You are a senior editor.
Summarize the following report into exactly **5 numbered points (1–5)** in markdown.
Each point should be one crisp sentence focusing on the most decision-relevant insights.

Report:
========
{report}
========

Return only the 5 numbered lines."""
)

# Assemble the summary chain:
# PromptTemplate -> ChatGoogleGenerativeAI -> StrOutputParser
summary_chain = summary_prompt | llm | StrOutputParser()


#this works
# final_answer  = summary_chain.invoke({"report": report_chain.invoke({"topic": "Obesity in Urban India"})})
# print(final_answer, "final_answer")


final_chain = report_chain | summary_chain
final_answer = final_chain.invoke({"topic": "Obesity in Urban India"})
print(final_answer, "final_answer")


1.  Urban India faces a rapidly escalating obesity epidemic, with prevalence rates now comparable to developed nations, affecting all demographics and expanding beyond major metros.
2.  The primary drivers are a shift towards Westernized, processed food diets, increasingly sedentary lifestyles, and urban environments lacking infrastructure for physical activity and healthy food access.
3.  This escalating obesity crisis is a major contributor to the rise of Non-Communicable Diseases (NCDs) like diabetes and cardiovascular issues, severely impacting public health.
4.  The economic burden is substantial, encompassing soaring healthcare costs for NCDs and significant productivity losses, threatening both household and national financial stability.
5.  Addressing this requires urgent, multi-sectoral interventions, including integrated urban planning, food environment regulation, school and workplace wellness programs, and improved data surveillance. final_answer
